In [44]:
import spacy
from spacy import displacy
from spacy.training import Example
from spacy.lang.en import English
import random
import json

temp_nlp = English()


labeled_data = []
with open(r"/home/zhenyuan/AttacKG/NLP/Doccano/admin.jsonl", "r") as read_file:
    for line in read_file:
        data = json.loads(line)
        labeled_data.append(data)

print('---Read Labeled Data(%d)!---' % len(labeled_data))

sample = "APT3 has used PowerShell on victim systems to download and run payloads after exploitation."

    # self.nlp.initialize(lambda: self.spacy_data)
# split training and testing set
# training_set = spacy_data[0:19]
# testing_set = spacy_data[20:-1]

---Read Labeled Data(27)!---


In [52]:
# nlp = spacy.blank("en")
# nlp = spacy.load("en_core_web_sm") # python -m spacy download en_core_web_sm
model_location = None
# model_location = "/home/zhenyuan/AttacKG/NLP/cti.model"
ner_labels = ["NetLoc", "APTFamily", "ExeFile", "ScriptsFile", "DocumentFile", "E-mail", "Registry", "File", "Vulnerability", "C2C", "SensInfo", "Service"]

if model_location is None:
    nlp = spacy.blank('en')
    print("---Created Blank 'en' Model---")
else:
    nlp = spacy.load(model_location)
    print("---Load Model: %s---" % model_location)

if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")

for label in ner_labels:
    ner.add_label(label)

if model_location is None:
    optimizer = nlp.begin_training()
else:
    optimizer = ner.create_optimizer()

---Created Blank 'en' Model---


In [50]:
# Data format converting
spacy_data = []
for entry in labeled_data:
    entities = []
    for e in entry['label']:
        entities.append((e[0], e[1], e[2]))
    spacy_data.append(Example.from_dict(nlp.make_doc(entry['data']), {"entities": entities}))
    # spacy_data.append((entry['data'], {"entities": entities}))
print(spacy_data)

[{'doc_annotation': {'cats': {}, 'entities': ['B-APTFamily', 'L-APTFamily', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'links': {}}, 'token_annotation': {'ORTH': ['Cobalt', 'Strike', 'can', 'use', 'known', 'credentials', 'to', 'run', 'commands', 'and', 'spawn', 'processes', 'as', 'a', 'domain', 'user', 'account', '.'], 'SPACY': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False], 'TAG': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], 'LEMMA': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], 'POS': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], 'MORPH': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], 'HEAD': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 'DEP': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], 'SENT_START': [1, 0, 0, 0, 0, 0, 0, 0, 

In [59]:
# Start training
print("---Start Training!---")
new_model_location = "/home/zhenyuan/AttacKG/NLP/new_cti.model"

# Loop
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):
    for itn in range(4):
        random.shuffle(spacy_data)
        losses = ()

        # Batch the examples
        for batch in spacy.util.minibatch(spacy_data, size=2):
            # Update the model
            nlp.update(batch, sgd=optimizer) #, drop=0.35, losses=losses
            # print('Losses', losses)

nlp.to_disk(new_model_location)
print("---Save Model to %s!---" % model_location)

---Start Training!---
---Save Model to None!---


In [57]:
#def test_model(sample = sample):
doc = nlp(sample)
displacy.render(doc, style='ent')

In [61]:
sample = "MCMD can use Registry Run Keys for persistence."
nlp = spacy.load(new_model_location)
# nlp = spacy.load("en_core_web_sm")
doc = nlp(sample)
displacy.render(doc, style = 'ent')

UsageError: Line magic function `%run('pwd')` not found.
